# References

1. https://pandas.pydata.org/docs/
1. https://numpy.org/doc/
1. https://matplotlib.org/stable/index.html
1. https://scikit-learn.org/stable/
1. https://seaborn.pydata.org/
1. https://pytorch.org/tutorials/
1. https://pytorch.org/vision/main/models/vision_transformer.html
1. https://medium.com/@brianpulfer/vision-transformers-from-scratch-pytorch-a-step-by-step-guide-96c3313c2e0c
1. Data Loading and Processing is based on CSE 676 Deep Learning Assignment 1 Part 1 submission by Nikhil Gupta